In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2 
import psycopg2.extras
import io
import getpass

In [2]:
# Data for the project comes from https://github.com/JeffSackmann/tennis_wta
path = 'https://raw.githubusercontent.com/JeffSackmann/tennis_wta/master'

In [3]:
# A list of the data for the WTA matches from 2010 to 2021 is created and then Concatenated into a DataFrame
df_list = [pd.read_csv(path + '/wta_matches_' + str(year) + '.csv') for year in range(2010,2021)]
df = pd.concat(df_list)
df.rename(columns = {'round':'rounds'}, inplace = True)
df.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2010-D001,Fed Cup WG R1: UKR vs ITA,Hard,4,D,20100206,1,201416,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,26.0,2020.0,18.0,2720.0
1,2010-D001,Fed Cup WG R1: UKR vs ITA,Hard,4,D,20100206,2,201355,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,3160.0,33.0,1680.0
2,2010-D001,Fed Cup WG R1: UKR vs ITA,Hard,4,D,20100206,3,201355,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,3160.0,26.0,2020.0
3,2010-D001,Fed Cup WG R1: UKR vs ITA,Hard,4,D,20100206,4,201212,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,18.0,2720.0,33.0,1680.0
4,2010-D002,Fed Cup WG R1: CZE vs GER,Hard,4,D,20100206,1,201394,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,66.0,950.0,46.0,1325.0


In [4]:
df.shape

(29252, 49)

In [5]:
df_wta_matches = df[['tourney_date', 'tourney_name', 'tourney_level', 'surface', 'rounds', 'best_of', 'winner_id', 'winner_name', 'winner_rank', 'loser_id', 'loser_name', 'loser_rank']]
df_wta_matches.insert(loc=0, column='index', value=np.arange(len(df_wta_matches)))
df_wta_matches.head()

,index,tourney_date,tourney_name,tourney_level,surface,rounds,best_of,winner_id,winner_name,winner_rank,loser_id,loser_name,loser_rank
0,0,20100206,Fed Cup WG R1: UKR vs ITA,D,Hard,RR,3,201416,Alona Bondarenko,26.0,201212,Francesca Schiavone,18.0
1,1,20100206,Fed Cup WG R1: UKR vs ITA,D,Hard,RR,3,201355,Flavia Pennetta,12.0,201435,Kateryna Bondarenko,33.0
2,2,20100206,Fed Cup WG R1: UKR vs ITA,D,Hard,RR,3,201355,Flavia Pennetta,12.0,201416,Alona Bondarenko,26.0
3,3,20100206,Fed Cup WG R1: UKR vs ITA,D,Hard,RR,3,201212,Francesca Schiavone,18.0,201435,Kateryna Bondarenko,33.0
4,4,20100206,Fed Cup WG R1: CZE vs GER,D,Hard,RR,3,201394,Anna Lena Groenefeld,66.0,201425,Lucie Safarova,46.0


In [6]:
# All WTA players Column header added.
df_players = pd.read_csv(path + '/wta_players.csv', names=['player_id', 'first_name', 'last_name', 'hand', 'birth_date', 'country'])
df_players.head()

,player_id,first_name,last_name,hand,birth_date,country
0,113190,Bobby,Riggs,U,NaN,USA
1,200000,X,X,U,19000000.0,UNK
2,200001,Martina,Hingis,R,19800930.0,SUI
3,200002,Mirjana,Lucic,R,19820309.0,CRO
4,200003,Justine,Henin,R,19820601.0,BEL


In [7]:
df_players.shape

(41743, 6)

In [8]:
# Current WTA rankings.  Column header added.
df_rankings = pd.read_csv(path + '/wta_rankings_current.csv', names=['date', 'ranking', 'id', 'points', 'unk'])
df_rankings.insert(loc=0, column='index', value=np.arange(len(df_rankings)))
df_rankings.head()

,index,date,ranking,id,points,unk
0,0,20210104,1,202458,8717,17
1,1,20210104,2,201594,7255,17
2,2,20210104,3,211768,5780,16
3,3,20210104,4,213550,5760,25
4,4,20210104,5,202494,5260,26


In [9]:
df_rankings_wta = df_rankings[['index', 'date', 'ranking', 'id']]
df_rankings_wta.head()

,index,date,ranking,id
0,0,20210104,1,202458
1,1,20210104,2,201594
2,2,20210104,3,211768
3,3,20210104,4,213550
4,4,20210104,5,202494


In [10]:
df_rankings_wta.shape

(44197, 4)

In [11]:
try:
    p = getpass.getpass()
except Exception as error:
    print('ERROR', error)

········


In [12]:
engine = create_engine('postgresql://postgres:' + p + '@localhost:5432/WTA')

# Connect to PostgreSQL server
dbConnection = engine.connect()

#Add matches, players and rankings DataFrames to postgres Database WTA.  Replace table if table already exists
df_wta_matches.to_sql('matches', con=engine, if_exists='replace',index=False)
df_players.to_sql('players', con=engine, if_exists='replace',index=False)
df_rankings_wta.to_sql('rankings', con=engine, if_exists='replace',index=False)

# Close the database connection
dbConnection.close();